### Rank Scores

In [1]:
import pandas as pd
import pandasql as ps

In [2]:
data = [[1, 3.5], [2, 3.65], [3, 4.0], [4, 3.85], [5, 4.0], [6, 3.65]]
Scores = pd.DataFrame(data, columns=['id', 'score']).astype({'id':'Int64', 'score':'Float64'})
print(Scores)

   id  score
0   1    3.5
1   2   3.65
2   3    4.0
3   4   3.85
4   5    4.0
5   6   3.65


In [14]:
Scores['rank'] = Scores['score'].rank(ascending=False, method='dense')
print(Scores[['score','rank']].sort_values('rank').reset_index(drop=True))

   score  rank
0    4.0   1.0
1    4.0   1.0
2   3.85   2.0
3   3.65   3.0
4   3.65   3.0
5    3.5   4.0


In [19]:
query = '''
select score,
dense_rank() over(
    order by score desc
) as rank
from Scores
'''

ps.sqldf(query,locals())

,score,rank
0,4.00,1
1,4.00,1
2,3.85,2
3,3.65,3
4,3.65,3
5,3.50,4


### Delete Duplicate Emails

In [102]:
data = [[1, 'john@example.com'], [2, 'bob@example.com'], [3, 'john@example.com']]
Person = pd.DataFrame(data, columns=['id', 'email']).astype({'id':'int64', 'email':'object'})
print(Person)

   id             email
0   1  john@example.com
1   2   bob@example.com
2   3  john@example.com


In [112]:
print(Person.drop_duplicates(subset ='email', keep = 'first', inplace = False))

   id             email
0   1  john@example.com
1   2   bob@example.com


In [108]:
query = '''
DELETE FROM Person
WHERE Id NOT IN (
    SELECT MIN(Id)
    FROM Person
    GROUP BY Email)
'''
ps.sqldf(query, locals()) # ps는 변경 작업 처리 적용이 힘듦

### Rearrange Products Table

In [114]:
data = [[0, 95, 100, 105], [1, 70, None, 80]]
Products = pd.DataFrame(data, columns=['product_id', 'store1', 'store2', 'store3'])
print(Products)

   product_id  store1  store2  store3
0           0      95   100.0     105
1           1      70     NaN      80


In [163]:
pd.melt(Products, id_vars = ['product_id'],
        value_vars = [col for col in Products.columns if col != 'product_id'],
        var_name = 'store', value_name = 'price').dropna(subset=['price']).sort_values(by='product_id').reset_index(drop=True)

,product_id,store,price
0,0,store1,95.0
1,0,store2,100.0
2,0,store3,105.0
3,1,store1,70.0
4,1,store3,80.0


In [172]:
df = pd.DataFrame({
    'A': ['A0', 'A1', 'A2'],
    'B': ['B0', 'B1', 'B2'],
    'C': [1, 2, 3],
    'D': [4, 5, 6]
})

print(df)

    A   B  C  D
0  A0  B0  1  4
1  A1  B1  2  5
2  A2  B2  3  6


In [176]:
melted = pd.melt(df, id_vars=['A', 'B'], 
                value_vars=['C', 'D'])

print(melted)

    A   B variable  value
0  A0  B0        C      1
1  A1  B1        C      2
2  A2  B2        C      3
3  A0  B0        D      4
4  A1  B1        D      5
5  A2  B2        D      6


In [183]:
query = '''
with StoreNames as (
    select 'store1' AS store_name
    union
    select 'store2'
    union
    select 'store3'
)

select
    p.Product_id,
    s.store_name as store,
    case 
        when s.store_name = 'store1' then p.store1
        when s.store_name = 'store2' then p.store2
        when s.store_name = 'store3' then p.store3
    end as price
from
    Products p
cross join
    StoreNames s
where
    case
        when s.store_name = 'store1' then p.store1
        when s.store_name = 'store2' then p.store2
        when s.store_name = 'store3' then p.store3
    end is not null;
'''

ps.sqldf(query, locals())

,product_id,store,price
0,0,store1,95.0
1,0,store2,100.0
2,0,store3,105.0
3,1,store1,70.0
4,1,store3,80.0
